# TMS tariff


This is a text cell. Start editing!




In [1]:
spark.catalog.listTables.show(false)

+------------------+--------+-----------+---------+-----------+
|name              |database|description|tableType|isTemporary|
+------------------+--------+-----------+---------+-----------+
|btl_ra_data       |default |null       |EXTERNAL |false      |
|raw_tarif_init    |default |null       |EXTERNAL |false      |
|raw_tarif_update  |default |null       |EXTERNAL |false      |
|stg_aufenthaltsart|default |null       |EXTERNAL |false      |
|stg_deckung       |default |null       |EXTERNAL |false      |
|stg_ecdetail      |default |null       |EXTERNAL |false      |
|stg_eckopf        |default |null       |EXTERNAL |false      |
|stg_ecprodukt     |default |null       |EXTERNAL |false      |
|stg_familie       |default |null       |EXTERNAL |false      |
|stg_gemeinde      |default |null       |EXTERNAL |false      |
|stg_leipkopf      |default |null       |EXTERNAL |false      |
|stg_leippos       |default |null       |EXTERNAL |false      |
|stg_schadenart    |default |null       

In [2]:
spark.table("default.raw_tarif_init").printSchema

root
 |-- _xmlns: string (nullable = true)
 |-- _xmlns:TMS: string (nullable = true)
 |-- _xmlns:xsi: string (nullable = true)
 |-- _xsi:schemaLocation: string (nullable = true)
 |-- header: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _exportedAt: timestamp (nullable = true)
 |    |-- _mandators: string (nullable = true)
 |    |-- _numberOfElements: long (nullable = true)
 |    |-- _tariffDescriptor: long (nullable = true)
 |-- initialLoad: struct (nullable = true)
 |    |-- _knownAt: timestamp (nullable = true)
 |    |-- tariffNodes: struct (nullable = true)
 |    |    |-- tariffNode: struct (nullable = true)
 |    |    |    |-- _businessDescriptor: string (nullable = true)
 |    |    |    |-- _nodeType: string (nullable = true)
 |    |    |    |-- _parent: long (nullable = true)
 |    |    |    |-- _parentNodeType: string (nullable = true)
 |    |    |    |-- _validFrom: date (nullable = true)
 |    |    |    |-- _validTo: date (nullable = true)
 |  

In [3]:
spark.table("default.raw_tarif_init").select("initialLoad.tariffNodes.tariffNode.*").show()

+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+
|_businessDescriptor|_nodeType|_parent|_parentNodeType|_validFrom|  _validTo|        descriptions|         tariffNodes|
+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+
|                  A|        C|    402|              R|2008-01-01|2999-12-31|{[{de, Verdauungs...|{{A01, C, A, C, 2...|
+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+



In the present small example are 6 levels of `tariffNodes.tariffNode`. The first 3 are structs, then the lower ones are arrays. <div>Let's explore them, starting with the 3. level.</div>

In [5]:
val df = spark.table("default.raw_tarif_init")
val dfl3 = df.withColumn("TarifNodes", explode($"initialLoad.tariffNodes.tariffNode.tariffNodes.tariffNode.tariffNodes.tariffNode")).select("TarifNodes.*")
dfl3.show()

+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+--------------------+
|_businessDescriptor|_nodeType|_parent|_parentNodeType|_validFrom|  _validTo|        descriptions|              galDat|         tariffNodes|
+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+--------------------+
|               A01A|        C|    A01|              C|2008-01-01|2999-12-31|{[{de, Stomatolog...|                null|{[{A01AA, C, A01A...|
|      9001570516763|        P|     XY|              C|2018-04-01|2999-12-31|{[{de, Sempermed ...|{{[{null, 9001570...|                null|
|      9001570516770|        P|     XY|              C|2018-04-01|2999-12-31|{[{de, Sempermed ...|{{[{null, 9001570...|                null|
|      9001570516787|        P|     XY|              C|2018-04-01|2999-12-31|{[{de, Sempermed ...|{{[{null, 9001570...|                null|
|      900157

In [6]:
val dfl4 = dfl3.filter("tariffNodes is not null")
            .withColumnRenamed("_businessDescriptor", "bD3")
            .withColumn("TarifNodes4", explode($"tariffNodes.tariffNode")).select("bD3", "TarifNodes4.*")
dfl4.show()

+----+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+
| bD3|_businessDescriptor|_nodeType|_parent|_parentNodeType|_validFrom|  _validTo|        descriptions|         tariffNodes|
+----+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+
|A01A|              A01AA|        C|   A01A|              C|2008-01-01|2999-12-31|{[{de, Kariesprop...|{[{A01AA01, C, A0...|
|A01A|              A01AB|        C|   A01A|              C|2008-01-01|2999-12-31|{[{de, Antiinfekt...|{[{A01AB02, C, A0...|
+----+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+



In [7]:
val dfl5 = dfl4.filter("tariffNodes is not null")
            .withColumnRenamed("_businessDescriptor", "bD4")
            .withColumn("TarifNodes5", explode($"tariffNodes.tariffNode")).select("bD3", "bD4", "TarifNodes5.*")
dfl5.show()

+----+-----+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+--------------------+
| bD3|  bD4|_businessDescriptor|_nodeType|_parent|_parentNodeType|_validFrom|  _validTo|        descriptions|              galDat|         tariffNodes|
+----+-----+-------------------+---------+-------+---------------+----------+----------+--------------------+--------------------+--------------------+
|A01A|A01AA|            A01AA01|        C|  A01AA|              C|2008-01-01|2999-12-31|{[{de, Natrium fl...|                null|{[{7680152190687,...|
|A01A|A01AA|            A01AA02|        C|  A01AA|              C|2008-01-01|2999-12-31|{[{de, Natrium mo...|                null|                null|
|A01A|A01AA|            A01AA03|        C|  A01AA|              C|2008-01-01|2999-12-31|{[{de, Olaflur}, ...|                null|{[{7610458026779,...|
|A01A|A01AA|            A01AA04|        C|  A01AA|              C|2008-01-01|2999-12-31|

In [8]:
val dfl6 = dfl5.filter("tariffNodes is not null")
            .withColumnRenamed("_businessDescriptor", "bD5")
            .withColumn("TarifNodes6", explode($"tariffNodes.tariffNode")).select("bD3", "bD4", "bD5", "TarifNodes6.*")
dfl6.show(false)

+----+-----+-------+-------------------+---------+-------+---------------+----------+----------+-------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|bD3 |bD4  |bD5    |_businessDescriptor|_nodeType|_parent|_parentNodeType|_validFrom|_validTo  |descriptions                                                                                                       |galDat                                                                                                                                                                                                                                                          

let's try flatten all in once (not yet unrolling all levels)

In [10]:
import pyspark
import pyspark.sql.functions as F

def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df


df = spark.table("default.raw_tarif_init")
df_flat = flatten_df(df.select("initialLoad.tariffNodes.tariffNode"))
df_flat.show(20,True)

+------------------------------+--------------------+------------------+--------------------------+---------------------+-------------------+-----------------------+----------------------+
|tariffNode__businessDescriptor|tariffNode__nodeType|tariffNode__parent|tariffNode__parentNodeType|tariffNode__validFrom|tariffNode__validTo|tariffNode_descriptions|tariffNode_tariffNodes|
+------------------------------+--------------------+------------------+--------------------------+---------------------+-------------------+-----------------------+----------------------+
|                             A|                   C|               402|                         R|           2008-01-01|         2999-12-31|   {[{de, Verdauungs...|  {{A01, C, A, C, 2...|
+------------------------------+--------------------+------------------+--------------------------+---------------------+-------------------+-----------------------+----------------------+



In [11]:
spark.table("default.raw_tarif_init").printSchema

root
 |-- _xmlns: string (nullable = true)
 |-- _xmlns:TMS: string (nullable = true)
 |-- _xmlns:xsi: string (nullable = true)
 |-- _xsi:schemaLocation: string (nullable = true)
 |-- header: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _exportedAt: timestamp (nullable = true)
 |    |-- _mandators: string (nullable = true)
 |    |-- _numberOfElements: long (nullable = true)
 |    |-- _tariffDescriptor: long (nullable = true)
 |-- initialLoad: struct (nullable = true)
 |    |-- _knownAt: timestamp (nullable = true)
 |    |-- tariffNodes: struct (nullable = true)
 |    |    |-- tariffNode: struct (nullable = true)
 |    |    |    |-- _businessDescriptor: string (nullable = true)
 |    |    |    |-- _nodeType: string (nullable = true)
 |    |    |    |-- _parent: long (nullable = true)
 |    |    |    |-- _parentNodeType: string (nullable = true)
 |    |    |    |-- _validFrom: date (nullable = true)
 |    |    |    |-- _validTo: date (nullable = true)
 |  

In [12]:
spark.table("default.stg_tarif_init").filter("tariffNodes is not null").show()

+--------------------+-------------------+---------+-------+---------------+----------+----------+--------------------+------+--------------------+
|          importFile|_businessDescriptor|_nodeType|_parent|_parentNodeType|_validFrom|  _validTo|        descriptions|galDat|         tariffNodes|
+--------------------+-------------------+---------+-------+---------------+----------+----------+--------------------+------+--------------------+
|file:/mnt/data/TM...|               A01A|        C|    A01|              C|2008-01-01|2999-12-31|{[{de, Stomatolog...|  null|{[{A01AA, C, A01A...|
+--------------------+-------------------+---------+-------+---------------+----------+----------+--------------------+------+--------------------+



# Tarif Update



In [14]:
val df_tarif = spark.table("default.stg_tarif_mod")
df_tarif.select("*").distinct().show(false)
df_tarif.select("galDat.galDatMain.*").distinct().show(false)

+----------------------------------------------------------------------+-------------------+---------+---------+---------------+----------+----------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
val ex1 = df_tarif.filter($"_businessDescriptor" === "7640143440498").filter($"_validTo" === "2999-12-31")
ex1.show(false)
ex1.select("galDat.galDatMain.*").show(false)

+----------------------------------------------------------------------+-------------------+---------+-------+---------------+----------+----------+-----------+--------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|importFile                                                            |_businessDescriptor|_nodeType|_parent|_parentNodeType|_validFrom|_validTo  |assignments|descriptions                          |galDat                                                                                                                                                                                                                                                                                          

In [16]:
val ex2 = df_tarif.filter($"_businessDescriptor" === "5016003781605").filter($"_validTo" === "2999-12-31")
ex2.show(false)
ex2.select("galDat.galDatMain.*").show(false)

+----------------------------------------------------------------------+-------------------+---------+---------+---------------+----------+----------+-----------+------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|importFile                                                            |_businessDescriptor|_nodeType|_parent  |_parentNodeType|_validFrom|_validTo  |assignments|descriptions                                                                              |galDat                                                                                                                                         

In [17]:
df_tarif.select("galDat.galDatMain._newInsuranceCode").groupBy($"_newInsuranceCode").count().sort("_newInsuranceCode").show()

+-----------------+-----+
|_newInsuranceCode|count|
+-----------------+-----+
|             null|  116|
|               10| 2379|
|               12|   72|
|               13|   55|
|               20|   67|
|               30| 1133|
|               32|   73|
|               33|    1|
|               34|  136|
|               80|   12|
|               81|    1|
|               99|   77|
+-----------------+-----+



In [18]:
df_tarif.filter($"galDat.galDatMain._newInsuranceCode" === 10).select("_businessDescriptor", "descriptions.description", "galDat.galDatMain.*").show(false)

+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+------+-----------+------------+----------+-------------+------------------------------------------+------------+------------------------+----+----------+----------+---------+--------+-----------+-------+--------+-----------------+---------+-----------+--------+-------------+-----------------+----------------+----------------------+-------------------+-------------+---------------------+--------------+---------------+--------------+--------+
|_businessDescriptor|description                                                                                                                                    |_VALUE|_assortKey1|_bsvCancDate|_bsvSLDate|_companyGln  |_companyName                              |_genericCode|_genericGroup           |_ggl|_grdFrCode|_groupCode|_hospCode|_insCode|_insCodeLim|_limPkt|_loaCode|_newInsuran